In [128]:
# TODO
# get the various paths that are taken from our machine to the destination for various VPN services.
# calculate the time in multiple cases and record the average time

target = "54.185.10.135"

In [151]:
# shanmukh's ip geolocation lookup
import json
from ip2geotools.databases.noncommercial import DbIpCity

def get_location(target):
    result = json.loads(DbIpCity.get(target, api_key='free').to_json())
    #print(result)
    return result['city'],result['country']


In [142]:
# TCP Traceroute
from scapy.layers.inet import traceroute
from scapy.layers.inet import traceroute_map
# ans,unans = answered,unanswered packets list
# They are of type TracerouteResult defined in scapy/inet.py
# each entry in ans is a 2-entry tuple = send,recv packet


ans,unans = traceroute(target, maxttl=60, verbose=0)
index = len(ans)
for i,v in enumerate(ans):
    if v[1].src == target:
        index = i
        break

# scapy parallelly sends requests for all TTL's and stores results from them.
# Removing redundant entries after the packet reaches destination
ans[0:index+1]

<mod Traceroute: TCP:0 UDP:0 ICMP:21 Other:0>

In [153]:
print("List of answered packets:")
print()
for s,d in ans[0:index+1]:
    src = s.sprintf("%-03s,IP.ttl%  %IP.dst%")
    city,country = get_location(d.src)
    dst = d.sprintf("%-15s,IP.src% ("+city+","+country+")")
    print(src+" ==> "+dst)

List of answered packets:

2    54.185.10.135 ==> 129.49.159.131  (Stony Brook,US)
3    54.185.10.135 ==> 129.49.7.114    (New York,US)
5    54.185.10.135 ==> 199.109.5.129   (New York,US)
6    54.185.10.135 ==> 199.109.7.166   (Syracuse (Downtown),US)
7    54.185.10.135 ==> 199.109.7.93    (Syracuse (Downtown),US)
8    54.185.10.135 ==> 199.109.7.194   (Syracuse (Downtown),US)
9    54.185.10.135 ==> 199.109.11.34   (Newark,US)
10   54.185.10.135 ==> 162.252.70.131  (Newark,US)
12   54.185.10.135 ==> 162.252.70.107  (Newark,US)
11   54.185.10.135 ==> 162.252.70.162  (Newark,US)
13   54.185.10.135 ==> 162.252.70.172  (Newark,US)
14   54.185.10.135 ==> 99.82.179.38    (Ashburn,US)
15   54.185.10.135 ==> 52.95.52.182    (Ashburn,US)
16   54.185.10.135 ==> 52.95.52.63     (Ashburn,US)
19   54.185.10.135 ==> 54.239.42.217   (Seattle (South Lake Union),US)
20   54.185.10.135 ==> 52.93.12.226    (Seattle (South Lake Union),US)
18   54.185.10.135 ==> 52.93.131.47    (Seattle (South Lake Union)

In [132]:
print("List of unanswered packets:")
print()
print("TTL  Source IP      Port")
for s in unans:
    print(s.sprintf("%-03s,IP.ttl%  %IP.dst%  {TCP:%ir,TCP.dport%}{UDP:udp%ir,UDP.dport%}"))

List of unanswered packets:

TTL  Source IP      Port
1    54.185.10.135  80
4    54.185.10.135  80
18   54.185.10.135  80
19   54.185.10.135  80
20   54.185.10.135  80
26   54.185.10.135  80
27   54.185.10.135  80
28   54.185.10.135  80
29   54.185.10.135  80
30   54.185.10.135  80


In [133]:
# In case its needed
from subprocess import Popen,PIPE
process = Popen(['cat', 'test.py'], stdout=PIPE, stderr=PIPE)
stdout, stderr = process.communicate()
print(stdout)

b''
